In [3]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import sys
sys.path.append("../../lib")
from metrics import levenshtein
import pickle

In [4]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish/CZ/"

In [5]:
output_folder = Path("../../data/cz")

In [6]:
files = sorted(os.listdir(folder))
len(files)

1

In [7]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

149

In [8]:
from multiprocessing import Pool

def extract(name):
    with open(name) as file:
        return file.readlines()
    
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128))
len(data)

  0%|          | 0/149 [00:00<?, ?it/s]

149

In [7]:
# data = []
# for f in tqdm(files):
#     with open(f) as file:
#         data.append(file.readlines())

data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(149, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"68 klů, kdežto potrawa ku zplozenj kosti slono...","@68 klů, kdežto potrawa ku zplozenj kosti slon...","68 klů, kdežto potrawa ku zplozenj kosti slon..."
1,Organisacni statut pro ústavy učitelské. 165 §...,@Organisacni statut pro ústavy učitelské. 165 ...,Organisační statut pro ústavy učitelské. 165 ...
2,207 ženského pokroku. Za jeho panováni popi uč...,@207 ženského pokroku. Za jeho panováni pop@@@...,207 ženského pokroku. Za jeho panování popřán...
3,10 o zemoplozj (geogonia) odkryl gsem takořka ...,@10 o zemoplozj (geogonia) odkryl gsem takořka...,10 o zemoplozj (geogonia) odkryl gsem takořka...
4,"- 189 — „Já jsem nemluvila o křesťauech,"" namí...","@- 189 — „Já jsem nemluvila o křesťauech,"" nam...","189 „Já jsem nemluvila o křesťauech,"" nam..."


In [8]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,149.000000,149.000000,149.000000
mean,1650.375839,1666.402685,1666.402685
std,1125.214493,1158.053008,1158.053008
min,4.000000,4.000000,4.000000
25%,1155.000000,1162.000000,1162.000000
50%,1362.000000,1365.000000,1365.000000
75%,1768.000000,1776.000000,1776.000000
max,6619.000000,7555.000000,7555.000000


In [9]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count      149.000000
mean       217.026916
std       2047.644134
min          0.328138
25%          1.158940
50%          1.858736
75%          3.927069
max      23900.000000
Name: cer, dtype: float64

In [10]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    149.000000
mean       6.016375
std       13.896263
min        0.328138
25%        1.155116
50%        1.846154
75%        3.809524
max       99.583333
Name: cer, dtype: float64

In [11]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder/"data/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

175


In [12]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder/"data/dev.pkl")
dev.shape

(5, 3)

In [13]:
train = data.drop(dev.index)
train.to_pickle(output_folder/"data/train.pkl")
train.shape

(144, 3)

In [23]:
train.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,144.000000,144.000000,144.000000
mean,1637.645833,1654.013889,1654.013889
std,1092.707150,1127.384323,1127.384323
min,4.000000,4.000000,4.000000
25%,1167.500000,1177.000000,1177.000000
50%,1362.500000,1370.000000,1370.000000
75%,1762.000000,1774.500000,1774.500000
max,6619.000000,7555.000000,7555.000000


In [14]:
dev.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,5.000000,5.000000,5.000000
mean,2017.000000,2023.200000,2023.200000
std,1996.575568,2004.362417,2004.362417
min,755.000000,756.000000,756.000000
25%,913.000000,914.000000,914.000000
50%,1103.000000,1108.000000,1108.000000
75%,1797.000000,1801.000000,1801.000000
max,5517.000000,5537.000000,5537.000000


In [15]:
levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     10.775038
std      12.046707
min       0.444198
25%       2.130493
50%       8.529742
75%      12.197393
max      30.573365
Name: cer, dtype: float64

In [16]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     10.568296
std      11.871602
min       0.444198
25%       2.116402
50%       8.315098
75%      11.823105
max      30.142677
Name: cer, dtype: float64

In [17]:
window_length = 100

In [18]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

(238322, 2)


,source,target
0,@— 445 — n A nyní bude tedy mou nejsvětější po...,445 @ „A nyní bude tedy mou nejsvětější po...
1,— 445 — n A nyní bude tedy mou nejsvětější pov...,445 @ „A nyní bude tedy mou nejsvětější pov...
2,445 — n A nyní bude tedy mou nejsvětější povi...,445 @ „A nyní bude tedy mou nejsvětější povi...
3,445 — n A nyní bude tedy mou nejsvětější povin...,445 @ „A nyní bude tedy mou nejsvětější povin...
4,45 — n A nyní bude tedy mou nejsvětější povinn...,45 @ „A nyní bude tedy mou nejsvětější povinn...


In [19]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,— 445 — n A nyní bude tedy mou nejsvětější pov...,445 @ „A nyní bude tedy mou nejsvětější po...
1,— 445 — n A nyní bude tedy mou nejsvětější pov...,445 @ „A nyní bude tedy mou nejsvětější pov...
2,445 — n A nyní bude tedy mou nejsvětější povi...,445 @ „A nyní bude tedy mou nejsvětější povi...
3,445 — n A nyní bude tedy mou nejsvětější povin...,445 @ „A nyní bude tedy mou nejsvětější povin...
4,45 — n A nyní bude tedy mou nejsvětější povinn...,45 @ „A nyní bude tedy mou nejsvětější povinn...


In [20]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(10121, 2)


,source,target
0,@OrganisaČni statut pro ústavy učitelské. 149 ...,Organisační statut pro ústavy učitelské. 149 ...
1,OrganisaČni statut pro ústavy učitelské. 149 O...,Organisační statut pro ústavy učitelské. 149 O...
2,rganisaČni statut pro ústavy učitelské. 149 O ...,rganisační statut pro ústavy učitelské. 149 O ...
3,ganisaČni statut pro ústavy učitelské. 149 O p...,ganisační statut pro ústavy učitelské. 149 O p...
4,anisaČni statut pro ústavy učitelské. 149 O po...,anisační statut pro ústavy učitelské. 149 O po...


In [21]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,OrganisaČni statut pro ústavy učitelské. 149 O...,Organisační statut pro ústavy učitelské. 149 ...
1,OrganisaČni statut pro ústavy učitelské. 149 O...,Organisační statut pro ústavy učitelské. 149 O...
2,rganisaČni statut pro ústavy učitelské. 149 O ...,rganisační statut pro ústavy učitelské. 149 O ...
3,ganisaČni statut pro ústavy učitelské. 149 O p...,ganisační statut pro ústavy učitelské. 149 O p...
4,anisaČni statut pro ústavy učitelské. 149 O po...,anisační statut pro ústavy učitelské. 149 O po...


In [22]:
train_aligned.to_pickle("../../data/train_aligned.pkl")
dev_aligned.to_pickle("../../data/dev_aligned.pkl")